# Retrieve ECG summaries from DICOM

### Requirements

In [ ]:
# Import required libraries
import os;
import numpy as np;
import pandas as pd;
from pydicom import dcmread
from pydicom.waveforms import multiplex_array

#Set options
np.set_printoptions(threshold = 500)

### Function for retrieving summary annotations from ECG

In [ ]:
# Add code for obtaining summary sequence annotations within ECG DICOM

# To be added to constants used by ECGProcess
SUMMARY = {'Summary'  : 'WaveformAnnotationSequence',
          }

# To be added to ECGProcess
def retrieve_summary_dict(dicom):

    #### The summary dictionary
    summary_dict = {}
    for t, s in SUMMARY.items():
        # if present in SUMMARY assign
        if hasattr(dicom, s) and s == 'WaveformAnnotationSequence':
            seq = getattr(dicom,s)
            # Extract summary data by looping over annotation sequence
            for i in range(0,len(seq)):
                if hasattr(seq[i], 'UnformattedTextValue'):
                    summary_dict['Summary_L'+str(i)] = seq[i].UnformattedTextValue
                elif hasattr(seq[i],'ConceptNameCodeSequence') and hasattr(seq[i],'NumericValue'): 
                    if hasattr(getattr(seq[i],'ConceptNameCodeSequence')[0],'CodeMeaning'): 
                        summary_dict[seq[i].ConceptNameCodeSequence[0].CodeMeaning] = seq[i].NumericValue
        elif hasattr(dicom, s):
            summary_dict[t] = getattr(dicom, s)  
        elif not hasattr(dicom, s) and s == 'WaveformAnnotationSequence':
            summary_dict[t] = 'No summary annotations present' 
        else:
            raise ValueError('`{0}` is not available.'.format(s))
            
    return(summary_dict)


### Running an example

In [ ]:
# Example summary extraction from ECG in DICOM

# Define dicom path
path_to_dicom = "E:/DataExchange/Hartcentrum/ECG_input/processed/temp/"

# Obtain ECG files in folder
ECG_files = list() 
for path, subdirs, files in os.walk(path_to_dicom):
    for filename in files:
            ECG_files.append(path+'/'+filename) 

# Obtain ECG summaries from multiple ECG files
for ECG_file in ECG_files[0:10]:
    # Read ECG from DICOM
    dicom_record = dcmread(ECG_file)
    
    # Obtain summary annotations ECG from DICOM
    summary = retrieve_summary_dict(dicom_record)
    print(pd.DataFrame(summary, index=[0]))

# run function to retrieve dictionary with summary specifications (after inclusion in ECGProces)  
    # dicom_file_path = list_data_paths()['example_dicom']
    # dicom_record    = ECG_DCM(path=dicom_file_path) 
    # dicom_record.summary